# Setup

Import a few common modules

In [3]:
#CODE for points = 1
# import sklearn, numpy, os
import numpy as np
import os
import sklearn

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard



# to make this notebook's output stable across runs
np.random.seed(42)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Vanishing/Exploding Gradients Problem

In [4]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [5]:
z = np.linspace(-5, 5, 200)

## Xavier and He Initialization

In [9]:
#CODE for points = 1
# set activation to relu and kernel initializer to he_normal
initializer = tf.keras.initializers.HeNormal()
keras.layers.Dense(10, activation="relu", kernel_initializer=initializer)

### Leaky ReLU

In [10]:
#CODE for points = 1

# remember α is the hyperparameter that defines how much the function “leaks”
# set the value of alpha, use the value typically set

def leaky_relu(z, alpha=0.001):
    return np.maximum(alpha*z, z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [11]:
#CODE for points = 1
#load MNIST dataset from keras

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11501568/11490434 [==============================] - 0s 0us/step


In [12]:
#CODE for points = 1

# initialize kernel_initializer to "he_normal" and activation function to softmax

tf.random.set_seed(42)
np.random.seed(42)
initializer = tf.keras.initializers.HeNormal()
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer=initializer),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer=initializer),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [13]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [47]:
history = model.fit(X_train_full, y_train_full, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.7781 - accuracy: 0.8971 - val_loss: 0.7782 - val_accuracy: 0.8954
Epoch 2/5
1875/1875 [==============================] - 20s 10ms/step - loss: 0.7768 - accuracy: 0.8996 - val_loss: 0.7574 - val_accuracy: 0.9076
Epoch 3/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.7763 - accuracy: 0.8988 - val_loss: 0.7528 - val_accuracy: 0.9098
Epoch 4/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.7739 - accuracy: 0.8996 - val_loss: 0.7557 - val_accuracy: 0.9040
Epoch 5/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.7736 - accuracy: 0.9006 - val_loss: 0.7585 - val_accuracy: 0.9094


Now look at what happens if we try to use the ReLU activation function instead:

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

In [16]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))

model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [17]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [48]:
history = model.fit(X_train_full, y_train_full, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.7724 - accuracy: 0.8999 - val_loss: 0.7652 - val_accuracy: 0.9008
Epoch 2/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.7718 - accuracy: 0.9010 - val_loss: 0.7527 - val_accuracy: 0.9094
Epoch 3/5
1875/1875 [==============================] - 18s 10ms/step - loss: 0.7721 - accuracy: 0.9010 - val_loss: 0.7492 - val_accuracy: 0.9118
Epoch 4/5
1875/1875 [==============================] - 19s 10ms/step - loss: 0.7704 - accuracy: 0.9019 - val_loss: 0.7526 - val_accuracy: 0.9052
Epoch 5/5
1875/1875 [==============================] - 17s 9ms/step - loss: 0.7706 - accuracy: 0.9022 - val_loss: 0.7566 - val_accuracy: 0.9108


Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

In [20]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [21]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_106 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_107 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [22]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [ ]:
#bn1.updates #deprecated

In [23]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [24]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 6ms/step - loss: 0.8619 - accuracy: 0.7335 - val_loss: 0.4687 - val_accuracy: 0.8716
Epoch 2/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.4346 - accuracy: 0.8734 - val_loss: 0.3494 - val_accuracy: 0.9038
Epoch 3/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3559 - accuracy: 0.8965 - val_loss: 0.3001 - val_accuracy: 0.9174
Epoch 4/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.3115 - accuracy: 0.9101 - val_loss: 0.2695 - val_accuracy: 0.9242
Epoch 5/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.2815 - accuracy: 0.9175 - val_loss: 0.2444 - val_accuracy: 0.9306


Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [25]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [26]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [27]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 6ms/step - loss: 1.1288 - accuracy: 0.6813 - val_loss: 0.6171 - val_accuracy: 0.8486
Epoch 2/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5778 - accuracy: 0.8548 - val_loss: 0.4354 - val_accuracy: 0.8920
Epoch 3/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4590 - accuracy: 0.8818 - val_loss: 0.3585 - val_accuracy: 0.9108
Epoch 4/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3954 - accuracy: 0.8957 - val_loss: 0.3152 - val_accuracy: 0.9184
Epoch 5/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.3552 - accuracy: 0.9033 - val_loss: 0.2840 - val_accuracy: 0.9252


# Faster Optimizers

## Momentum optimization

In [28]:
#CODE for points = 1
# initialize lr and momentum to typical values

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

## Nesterov Accelerated Gradient

In [30]:
#CODE for points = 1
# initialize lr and momentum to typical values. Set nesterov so that it is used

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov="true")

# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [31]:
#CODE for points = 0.5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))

In [37]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/
# nadam optimizer

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=tf.keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="Nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_full, y_train_full, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 15s 8ms/step - loss: 1.2728 - accuracy: 0.8762 - val_loss: 0.8163 - val_accuracy: 0.8884
Epoch 2/2
1719/1719 [==============================] - 14s 8ms/step - loss: 0.8185 - accuracy: 0.8855 - val_loss: 0.7755 - val_accuracy: 0.9016


In [38]:
#CODE for points = .5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=tf.keras.regularizers.l2(0.01))

In [39]:
#CODE for points = 1

# activation function initialized as softmax
# nadam optimizer


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="Nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_full, y_train_full, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))


Epoch 1/2
1719/1719 [==============================] - 15s 8ms/step - loss: 1.2927 - accuracy: 0.8760 - val_loss: 0.8174 - val_accuracy: 0.8896
Epoch 2/2
1719/1719 [==============================] - 13s 8ms/step - loss: 0.8206 - accuracy: 0.8842 - val_loss: 0.7763 - val_accuracy: 0.8998
